In [ ]:
## THIS IS A SCRIPT TO OPEN HOURLY ERA5 NETCDF AND CALCULATE DAILY HEATFLUX
# AND SAVE DAILY OUTPUT IN ITS OWN NEW NETCDF.
#AFTER THIS, USE GDAL TO CONVERT TO GEOTIFF, CHANGE RESOLUTION, PROJECTION AND ALIGNMENT. AND QGIS FOR EXTENT
#THEN PROCESS TO ICE PRODUCTION IN HEATFLUX2ICEPRODUCTION SCRIPT

#!!!
#TO CHANGE: -- PATH TO ERA5 NETCDF
           #-- PATH TO OUTPUT NETCDF 


#Script to open netcdf and plot
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cartopy #Instead of from mpl_toolkits.basemap import Basemap "conda install -c conda-forge cartopy"
from netCDF4 import Dataset #"conda install -c anaconda netcdf4"
#from scipy.io import netcdf -- Alternative to netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#hourly data
path = '.nc'#Path to ERA file


from mpl_toolkits.axes_grid1 import make_axes_locatable

data = Dataset(path)
data.variables

In [ ]:
import xarray as xr                                                    # import xarray library. Necessary for daily

##GET LATS, LONS AND TIME
lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
#wind = data.variables['si10'][:,:,:] #wind
#u = data.variables['u10'][:,:,:]
#v = data.variables['v10'][:,:,:]

##SET UP TO CONVERT HOURLY TO DAILY
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)

##GET DAILY WIND SPEED -- same as other wind stuff
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]
u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 
windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)

#Extract data
#data.variables.keys() - check data in netcdf
DewPoint2m = tp_daily.variables['d2m'][:,:,:]
DewPoint2m = DewPoint2m.values

Temp2m = tp_daily.variables['t2m'][:,:,:]
Temp2m = Temp2m.values

SurfPres = tp_daily.variables['sp'][:,:,:]
SurfPres = SurfPres.values

InSol = tp_daily.variables['ssrd'][:,:,:] #'ssrd' = 'surface solar radiation downwards'
InSol = InSol.values
InSol = InSol*24 #bc want total throughout day. 
InTherm = tp_daily.variables['strd'][:,:,:]
InTherm = InTherm.values
InTherm = InTherm*24 #bc want total throughout day. 


stef = 5.67*(10**-8)
T0 = 273.15 - 0.0137 - 0.05199*34 - 0.00007225*34**2 #salinity of 34.0 from Bett(2020)
OutTherm = 0.99 * stef * (T0**4)


SensHeat = 1.3 * 1004 * 0.00144 * windspeed * (Temp2m - T0)


Lv = 2.51 * (10**6) # Lv = [2.501 − 0.00237(Ts − 273.15)] × 106 -- would change if use different salinity to get Ts/T0
rea = 611.21 * 10**(9.8094*(DewPoint2m - 273.15)/(DewPoint2m + 0.71))
es  = 611.21 * 10**(9.8094*(T0         - 273.15)/(T0         + 0.71))
LatHeat = (0.622 * 1.3 * Lv * 0.00144 * windspeed * (rea - es))/SurfPres

##CALCULATE NET HEAT FLUX USING ALL CLIMATE VARIABLES
#α= albedo of open water = 0.06 
NetHeat = (1-0.06)*InSol + InTherm - OutTherm + SensHeat + LatHeat


In [ ]:
## Write to netcdf
#https://unidata.github.io/python-training/workshop/Bonus/netcdf-writing/
import netCDF4 as nc
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = Dataset('_2017.nc',mode='w',format='NETCDF4_CLASSIC') #define path and filename
print(ncfile)


lat_dim = ncfile.createDimension('latitude', 19)     # latitude axis. Check shape NetHeat.shape - day/lat/lon
lon_dim = ncfile.createDimension('longitude', 85)    # longitude axis
time_dim = ncfile.createDimension('DAY', None)



ncfile.title='My heatflux data, 2020 daily'

latitude = ncfile.createVariable('latitude', np.float32, ('latitude',))
longitude = ncfile.createVariable('longitude', np.float32, ('longitude',))
DAY = ncfile.createVariable('DAY', np.float32, ('DAY',)) #this had day as typo on left before
HF = ncfile.createVariable('daily heat flux wm2',np.float64,('DAY','latitude','longitude')) #I put a slash in this name and it gave me an error!



In [ ]:
latitude[:] = lats
longitude[:] = lons
HF[:,:,:] = NetHeat

print(ncfile)
# close the Dataset.
ncfile.close(); print('Dataset is closed!')

In [ ]:
## Then in gdal (Mac terminal)

#add projection info, reproject to NSIDC so match SIC data, match resolution to SIC data, align pixels to SIC


#gdal_translate -of GTiff -a_srs EPSG:4326 HeatFluxOct_B.nc HeatFluxTif_cond.tiff
#gdalwarp -t_srs EPSG:3976 -tr 3125 -3125 -te -1810937.500 -1445312.500 -1064062.500 -554687.500 HeatFluxTif_cond.tiff HeatFluxTif_cond2.tiff 
#^ xmin xmax ymin ymax of the SIC


#Then in QGIS (issue in GDAL directly):

#Clip by extent to match SIC.
#Make sure Match extent of clipped raster to extent of mask layer is selected.